In [39]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
from pokemon_card_generator.data import card_data

cards_df = card_data.get_card_data()

In [50]:
# remove light from name
cards_df.name = cards_df.name.apply(lambda x: x.replace('Light ', ''))

In [51]:
# remove dark from name
cards_df.name = cards_df.name.apply(lambda x: x.replace('Dark ', ''))

In [52]:
[item for item in cards_df.name]

['Alakazam',
 'Blastoise',
 'Chansey',
 'Charizard',
 'Clefairy',
 'Gyarados',
 'Hitmonchan',
 'Machamp',
 'Magneton',
 'Mewtwo',
 'Nidoking',
 'Ninetales',
 'Poliwrath',
 'Raichu',
 'Venusaur',
 'Zapdos',
 'Beedrill',
 'Dragonair',
 'Dugtrio',
 'Electabuzz',
 'Electrode',
 'Pidgeotto',
 'Arcanine',
 'Charmeleon',
 'Dewgong',
 'Dratini',
 "Farfetch'd",
 'Growlithe',
 'Haunter',
 'Ivysaur',
 'Jynx',
 'Kadabra',
 'Kakuna',
 'Machoke',
 'Magikarp',
 'Magmar',
 'Nidorino',
 'Poliwhirl',
 'Porygon',
 'Raticate',
 'Seel',
 'Wartortle',
 'Abra',
 'Bulbasaur',
 'Caterpie',
 'Charmander',
 'Diglett',
 'Doduo',
 'Drowzee',
 'Gastly',
 'Koffing',
 'Machop',
 'Magnemite',
 'Metapod',
 'Nidoran ♂',
 'Onix',
 'Pidgey',
 'Pikachu',
 'Poliwag',
 'Ponyta',
 'Rattata',
 'Sandshrew',
 'Squirtle',
 'Starmie',
 'Staryu',
 'Tangela',
 'Voltorb',
 'Vulpix',
 'Weedle',
 'Clefable',
 'Electrode',
 'Flareon',
 'Jolteon',
 'Kangaskhan',
 'Mr. Mime',
 'Nidoqueen',
 'Pidgeot',
 'Pinsir',
 'Scyther',
 'Snorlax',
 '

In [79]:
# remove trainer's names from pokemon name
trainer_names = [
    "Blaine's ", "Brock's", "Erika's ", "Lt. Surge's ", "Misty's ",
    "Rocket's ", "Sabrina's ", "Giovanni's ", "Koga's "
]

for trainer in trainer_names:
   cards_df.name = cards_df.name.apply(lambda x: x.replace(trainer, '')) 